Kd Analysis
===

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import glob
import numpy as np
import matplotlib.pyplot as plt
import tifffile
import random
import itertools
from collections import defaultdict, Counter, namedtuple
from IPython.display import HTML, Image
from champ import misc, intensity, intensity_array
from Bio import SeqIO
import time
from JSAnimation import IPython_display
from matplotlib import animation
import yaml

# Run Specific Params

In [ ]:
date = ''
target_sequence_filename = "/shared/targets.yml"
Imin_name = 'Imin_const'
Imax_name = 'Imax_adjusted'
inten_fmt = 'LDA'
TA_calibration_strategy = 'neg_control'  # ['pam', 'neg_control', 'all_possible']
ABA_zero_strategy = 'neg_control'  # neg_control or max_concentration

dname = '_'.join([inten_fmt, Imin_name, Imax_name, TA_calibration_strategy, ABA_zero_strategy])
fig_dir = 'analysis/figs'
results_dir = 'analysis/results'

with open(target_sequence_filename) as f:
    targets = yaml.load(f)
    
custom_results_dir = os.path.join('results', date, 'custom')
custom_fig_dir = os.path.join('figs', date, 'custom')
inten_array_fpath = os.path.join(custom_results_dir, 'LDA_intensity_scores.txt')
print 'Date:', date

Load Data
===

In [ ]:
ExperimentStats = namedtuple('ExperimentStats', 'date project_name target_name')
runs = [
    ExperimentStats(date, project_name, target_name) 
    for date, project_name, target_name in
    [
        ('', '', ''), 
    ]
]

In [ ]:
target_by_run = {run: targets[run.target_name] for run in runs}

In [ ]:
custom_results_dir = {run: os.path.join('results', run.date, 'custom')
                      for run in runs}

target_len = len(target_by_run[runs[0]])
print 'Target Length:', target_len
print
for run in runs:
    print 'Target', run.target_name
    print 'Date:', run.date
    print 'Results dir:', custom_results_dir[run]
    assert os.path.isdir(custom_results_dir[run])
    print

## Load Data

In [ ]:
Kds, Kd_error = {run: {} for run in runs}, {run: {} for run in runs}
ABAs, ABA_error = {run: {} for run in runs}, {run: {} for run in runs}
ddGs, ddG_error = {run: {} for run in runs}, {run: {} for run in runs}
neg_control_target_by_run = {}
max_concentration_by_run = {}
for run in runs:
    print run.date, run.target_name
    fname = 'LDA_Imin_const_Imax_adjusted_Kds_and_ABAs.txt'
    fpath = os.path.join(custom_results_dir[run], fname)
    with open(fpath) as f:
        line = next(f)
        assert line.startswith('# Target:')
        target = line.strip().split(': ')[1]
        print target
        line = next(f)
        assert line.startswith('# Neg Control')
        neg_control_target_by_run[run] = line.strip().split(': ')[1]
        line = next(f)
        assert line.startswith('# Concentration')
        line = next(f)
        while not line.startswith('#'):
            max_concentration_by_run[run] = float(line.strip().split()[0])
            line = next(f)
        assert line.startswith('# Seq')
        for line in f:
            if line.startswith('#'):
                continue
            words = line.strip().split()
            seq = words[0]
            assert seq not in Kds[run], seq
            Kd, Kd_err, ABA, ABA_err = map(float, words[1:])
            Kds[run][seq] = Kd
            Kd_error[run][seq] = Kd_err
            ABAs[run][seq] = ABA
            ABA_error[run][seq] = ABA_err
    ddGs[run] = {seq: ABAs[run][target] - ABA for seq, ABA in ABAs[run].items()}

    ddG_error[run] = ABA_error[run]

In [ ]:
flabpal = {'blue': (.365, .647, .855),
           'yellow': (.871, .812, .247),
           'green': (.376, .741, .408),
           'red': (.945, .345, .329)}

In [ ]:
bases = 'ACGT'
color_given_base = dict(A=flabpal['blue'], C=flabpal['yellow'], G=flabpal['green'], T=flabpal['red'])

In [ ]:
IA = intensity_array.IntensityArray()
intensity_filename = os.path.join('results', date, 'custom', 'LDA_intensity_scores.txt')
IA.parse_intensities_file(intensity_filename)

def get_cluster_count_range(seq):
    cluster_counts = []
    for lol in IA.intensity_lol_given_seq[seq]: 
        cluster_counts.append(len([i for i in lol if i is not None]))
    return min(cluster_counts), max(cluster_counts)

for run in runs:
    target = targets[run.target_name]
    fig, ax = plt.subplots(figsize=(15, 5))
    idxs = np.arange(len(target))
    w = 0.5
    cluster_counts = {}
    for j, c in enumerate(bases):
        cluster_counts[c] = {}
        loc_ddGs = []
        loc_ddG_error = []
        ticks = []
        for i, t in enumerate(target):
            seq = target[:i] + c + target[i+1:]
            min_clusters, max_clusters = get_cluster_count_range(seq)
            cluster_counts[c][i] = min_clusters
            
            if seq in ddGs[run]:
                loc_ddGs.append(ddGs[run][seq])
                loc_ddG_error.append(ddG_error[run][seq])
                ticks.append(idxs[i])
                
        ticks = np.array(ticks)
        ax.bar(ticks - w/2.0 + w*j/4.0, loc_ddGs, 
               width=w/4.0, yerr=loc_ddG_error, 
               color=color_given_base[c], error_kw=dict(ecolor='k', alpha=0.6), label=c)
    ax.xaxis.grid(False)
    ax.set_xlim((-0.5, len(target)-0.5))
    ax.set_xticks(range(len(target)))
    ax.set_xticklabels(target)

    ylim = ax.get_ylim()
    for i, c in enumerate(target):
        ax.fill_between([i-0.5, i+0.5], [ylim[0]]*2, [ylim[1]]*2, color=color_given_base[c], alpha=0.14)
    ax.set_ylim(ylim)
    for i, _ in enumerate(target):
        y_offset = 0.05
        ax.text(i-w/1.3, ylim[1]-y_offset, "N=")
        for base in bases:
            y_offset += 0.05
            ax.text(i-w/1.3, ylim[1]-y_offset, cluster_counts[base][i], fontdict={'color': color_given_base[base]})
    fs = 16
    ax.set_title('Single Mismatch Penalties'.format(r=run), fontsize=fs)
    ax.set_xlabel('Target {r.target_name} Reference Sequence (Background Color)'.format(r=run), fontsize=fs)
    ax.set_ylabel(r'$ABA (K_{B}T)$', fontsize=fs)
    ax.legend(loc='right')
    ax.xaxis.set_ticks_position('none') 
    fig.savefig(os.path.join(fig_dir, 'doped_ham1.eps'))
    print

## Looks like it needs a model

This looks very much like there are specific transition penalties from reference to mismatch bases with position-specific weighting of those penalties. Let $T$ be the 4-by-4 transition matrix, $p$ be the position specific weights, and for sequence $i$ let $S_i$ be the 35-by-16 indicator matrix of position-by-transition. That is, given $S_i$, one can reconstruct both the reference and given sequences. Then our model is:

$$
    \Delta \Delta G = (p \otimes vec(T)) : S_i
$$

where $vec(T)$ is the natural flattening of $T$, $\otimes$ is the outer product, and $:$ is the Frobenius matrix inner product.

### Consider separately

In [ ]:
import scipy.sparse
import scipy.optimize

In [ ]:
def single_ham_seqs(ref):
    seqs = []
    for i, ref_base in enumerate(ref):
        for mm_base in bases.replace(ref_base, ''):
            seqs.append(ref[:i] + mm_base + ref[i+1:])
    return seqs

In [ ]:
def single_ham_mutation_name(ref, seq):
    found = False
    for i, (ref_base, seq_base) in enumerate(zip(ref, seq)):
        if ref_base != seq_base:
            assert not found, '%s\n%s' % (ref, seq)
            found = True
            name = '%s%d%s' % (ref_base, i, seq_base)
    return name

In [ ]:
M = {}
for run in runs:
    target = targets[run.target_name]
    transition_ddGs = defaultdict(list)
    flipout_idxs = []
    for seq in single_ham_seqs(target):
        name = single_ham_mutation_name(target, seq)
        transition = (name[0], name[-1])
        location = int(name[1:-1])
        if location in flipout_idxs:
            continue
        transition_ddGs[transition].append(ddGs[run][seq])

    M[run] = np.zeros((4, 4))
    for i, c1 in enumerate(bases):
        for j, c2 in enumerate(bases):
            if i != j:
                M[run][i, j] = np.average(transition_ddGs[(c1, c2)])

    fig, ax = plt.subplots(figsize=(8, 6))
    ms = ax.matshow(M[run], cmap=plt.get_cmap('Reds'))
    ax.set_ylabel('From')
    ax.set_xlabel('To')
    ax.set_xticks(range(4))
    ax.set_yticks(range(4))
    ax.set_xticklabels(bases)
    ax.set_yticklabels(bases)
    ax.xaxis.set_ticks_position('bottom')
    ax.set_title('Target {r.target_name} Average Mismatch Penalties'.format(r=run))

    cbar = plt.colorbar(ms)
    cbar.set_label(r'$\Delta \Delta G \left(k_B T\right)$', fontsize=fs)
    print

In [ ]:
def sequence_transition_vector(ref, seq):
    v = np.zeros((target_len * 12,))
    for i, (ref_base, seq_base) in enumerate(zip(ref, seq)):
        if ref_base != seq_base:
            idx = 12*i + 3*bases.index(ref_base) + bases.index(seq_base) \
                  - int(bases.index(seq_base) > bases.index(ref_base))
            v[idx] = 1
    return v

In [ ]:
A, data = {}, {}
for run in runs:
    target = targets[run.target_name]
    rows, cols, tmp_data = [], [], []
    for i, seq in enumerate(single_ham_seqs(target)):
        tmp_data.append(ddGs[run][seq])
        v = sequence_transition_vector(target, seq)
        for j, val in enumerate(v):
            if val:
                rows.append(i)
                cols.append(j)
    A[run] = scipy.sparse.coo_matrix((np.ones((len(rows),)), (rows, cols)), 
                                     shape=(len(single_ham_seqs(target)), target_len*12), dtype=np.int8)
    A[run] = A[run].tocsr()
    data[run] = np.array(tmp_data)

In [ ]:
res = {}
for run in runs:
    def score_free_transition(params):
        return np.linalg.norm(A[run].dot(np.outer(params[:target_len], params[target_len:]).flatten()) - data[run])

    x0 = np.ones((target_len+12,))
    for i in range(4):
        for j in range(3):
            if j < i:
                val = M[run][i, j]
            else:
                val = M[run][i, j+1]
            x0[target_len + 3*i + j] = val
    res[run] = scipy.optimize.minimize(score_free_transition, x0, method='Powell')

In [ ]:
pred = {run: A[run].dot(np.outer(res[run].x[:target_len], res[run].x[target_len:]).flatten())
        for run in runs}

In [ ]:
for run in runs:
    target = targets[run.target_name]
    
    fig = plt.figure(figsize=(15, 8))
    ax = plt.subplot2grid((2, 3), (0, 0), colspan=3)

    idxs = np.arange(len(target))
    w = 0.7

    all_idxs = []
    all_pred = []
    sin_hams = single_ham_seqs(target)
    for j, c in enumerate(bases):
        loc_ddGs = []
        loc_ddG_error = []
        loc_pred = []
        for i, t in enumerate(target):
            seq = target[:i] + c + target[i+1:]
            loc_ddGs.append(ddGs[run][seq])
            loc_ddG_error.append(ddG_error[run][seq])
            if seq != target:
                loc_pred.append(pred[run][sin_hams.index(seq)])
            else:
                loc_pred.append(0)
        ax.bar(idxs - w/2.0 + w*(j/4.0), loc_ddGs, 
               width=w/4.0, yerr=loc_ddG_error, 
               color=color_given_base[c], error_kw=dict(ecolor='k', alpha=0.6), label=c)

        all_idxs = np.r_[all_idxs, idxs - w/2.0 + w*(j)/4.0]
        all_pred += loc_pred

    #ax.bar(all_idxs, all_pred, width=w/4.0, color='k', alpha=0.3)
    ax.xaxis.grid(False)
    ax.set_xlim((-0.5, len(target)-0.5))
    ax.set_xticks(range(len(target)))
    ax.set_xticklabels(target)

    ylim = ax.get_ylim()
    for i, c in enumerate(target):
        ax.fill_between([i-0.5, i+0.5], [ylim[0]]*2, [ylim[1]]*2, color=color_given_base[c], alpha=0.07)
    ax.plot([target_len - 3.5]*2, ylim, 'k', alpha=0.8, linewidth=0.7)
    ax.set_ylim(ylim)

    fs = 16
    ax.set_title('{r.date} Target {r.target_name} Single Mismatch ABAs'.format(r=run), fontsize=fs)
    ax.set_xlabel('Target {} Reference Sequence'.format(run.target_name), fontsize=fs)
    ax.set_ylabel(r'$ABA (K_{B}T)$', fontsize=fs)
    ax.legend(loc='best')

    # Transition weights
    ax = plt.subplot2grid((2, 3), (1, 1))
    Mpred = np.zeros((4, 4))
    for i in range(4):
        for j in range(3):
            val = res[run].x[target_len + 3*i + j]
            if j < i:
                Mpred[i, j] = val
            else:
                Mpred[i, j+1] = val
    sum_M = Mpred.sum()
    Mpred /= sum_M


    ms = ax.matshow(Mpred, cmap=plt.get_cmap('Reds'))
    ax.grid(False)
    ax.set_ylabel('From')
    ax.set_xlabel('To')
    ax.set_xticks(range(4))
    ax.set_yticks(range(4))
    ax.set_xticklabels(bases)
    ax.set_yticklabels(bases)
    ax.xaxis.set_ticks_position('bottom')
    ax.set_title('Transition Penalties')

    cbar = plt.colorbar(ms)
    cbar.set_label(r'$k_B T$', fontsize=10)

    # Position Weights
    ax = plt.subplot2grid((2, 3), (1, 0))
    ax.plot(range(-2, target_len - 2), sum_M * res[run].x[:target_len])
    ax.set_xlabel('Position (PAM < 0, Target > 0)')
    ax.set_ylabel('Weight')
    ax.set_title('Position Weights')
    ticks = [-2, 0, 1] + range(6, target_len - 2, 6)
    labels = [-3, -1] + ticks[2:]
    ax.set_xticks(ticks)
    ax.set_xticklabels(labels)
    ax.set_xlim((-2, target_len - 3))

    # Predicted v Measures
    ax = plt.subplot2grid((2, 3), (1, 2))
    ddG_vals = [ddGs[run][seq] for seq in sin_hams]
    ax.plot(ddG_vals, pred[run], '.')
    ax.set_xlabel(r'Measured $\Delta \Delta G$ ($k_B T$)')
    ax.set_ylabel(r'Fit ABA ($k_B T$)')
#    ax.text(0, 10, '$R^2 = %.2f$' % R2(ddG_vals, pred[run]), fontsize=16, va='center')
    ax.set_title('Fit vs. Measured ABA')
    ax.set_aspect(1)

#    fig.savefig(os.path.join(custom_fig_dir, 'position_transition_model.pdf'))
    print

In [ ]:
from scipy.stats import pearsonr

In [ ]:
fig = plt.figure(figsize=(15, 12))

for row, run in enumerate(runs):
    target = targets[run.target_name]
    sin_hams = single_ham_seqs(target)
    Mpred = np.zeros((4, 4))
    for i in range(4):
        for j in range(3):
            val = res[run].x[target_len + 3*i + j]
            if j < i:
                Mpred[i, j] = val
            else:
                Mpred[i, j+1] = val
    avg_M = np.average([xx for xx in Mpred.flatten() if xx > 0])
    Mpred /= avg_M

    
    # Position Weights
    ax = plt.subplot2grid((3, 4), (row, 0), colspan=2)
    ax.plot(range(target_len), -avg_M * res[run].x[:target_len])
    ax.set_xlabel('Position')
    ax.set_ylabel('Penalty ($k_B T$)')
    #ticks = [-2, 0, 1] + range(6, target_len - 2, 6)
    #labels = [-3, -1] + ticks[2:]
    #ax.set_xticks(ticks)
    #ax.set_xticklabels(labels)
    ax.set_xticks(range(0, target_len, 5))
    xlim = (0, target_len-1)
    ax.plot(xlim, [0]*2, '--', color='grey', zorder=-1)
    ax.set_xlim(xlim)
    ylim = ax.get_ylim()
    #ax.fill_between([xlim[0], 0.5], ylim[0], ylim[1], color=0.8*np.ones((3,)), zorder=-10)
    ax.grid(False)
    ax.set_axis_bgcolor('white')
    ax.set_title('{} Target {}'.format(run.date, run.target_name))

    # Transition weights
    ax = plt.subplot2grid((3, 4), (row, 2))
    mn = min(xx for xx in Mpred.flatten() if xx > 0)
    mx = max(xx for xx in Mpred.flatten() if xx > 0)
    mx_diff = max(abs(1-mn), abs(mx-1))
    Mpred[Mpred == 0] = None
    ms = ax.matshow(Mpred, cmap=plt.get_cmap('coolwarm'), vmin=1-mx_diff, vmax=1+mx_diff)
    ax.grid(False)
    ax.set_ylabel('From')
    ax.set_xlabel('To')
    ax.set_xticks(range(4))
    ax.set_yticks(range(4))
    ax.set_xticklabels(bases)
    ax.set_yticklabels(bases)
    ax.xaxis.set_ticks_position('bottom')

    cbar = plt.colorbar(ms)
    cbar.set_label(r'Weight', fontsize=10)

    # Predicted v Measures
    ax = plt.subplot2grid((3, 4), (row, 3))
    ddG_vals = [-ddGs[run][seq] for seq in sin_hams]
    pred_vals = [-p for p in pred[run]]
    ax.plot(ddG_vals, pred_vals, '.')
    ax.set_xlabel(r'$\Delta ABA_{measured}$ ($k_B T$)')
    ax.set_ylabel(r'$\Delta ABA_{fit}$ ($k_B T$)')
#    ax.text(0, 10, '$R^2 = %.2f$' % R2(ddG_vals, pred[run]), fontsize=16, va='center')
    ax.set_aspect(1)
    xlim, ylim = ax.get_xlim(), ax.get_ylim()
    r, pval = pearsonr(ddG_vals, pred_vals)
    ax.text((3*xlim[0] + xlim[1])/4, (ylim[0] + 3*ylim[1])/4, '$r=%.2f$' % r, fontsize=16, ha='center', va='center')
    ax.grid(False)
    ax.set_axis_bgcolor('white')

fig.savefig(os.path.join(fig_dir, 'position_transition_models.png'), dpi=300)
fig.savefig(os.path.join(fig_dir, 'position_transition_models.eps'))
print

# All together

## Calibrate Data

In [ ]:
ref_run = runs[0]

def find_calibration_divisors(strategy='pam'):
    assert strategy in ['pam', 'neg_control', 'all_possible']
    divisor = {}

    ref_target = targets[ref_run.target_name]

    cutoffs = {}
    for run in runs:
        single_mm_ddGs = [ddGs[run][seq] for seq in single_ham_seqs(target_by_run[run])]
        cutoffs[run] = 0.8 * max(single_mm_ddGs)

    for other_run in runs:
        other_target = targets[other_run.target_name]
        ref_vals, other_vals = [], []
        if strategy == 'all_possible':
            for i in range(target_len):
                for mm_base in bases.replace(ref_target[i], ''):
                    if ref_target[i] != other_target[i]:
                        continue
                    ref_seq = ref_target[:i] + mm_base + ref_target[i+1:]
                    other_seq = other_target[:i] + mm_base + other_target[i+1:]

                    ref_ddG = ddGs[ref_run][ref_seq]
                    other_ddG = ddGs[other_run][other_seq]

                    if ref_ddG < cutoffs[ref_run] and other_ddG < cutoffs[other_run]:
                        ref_vals.append(ref_ddG)
                        other_vals.append(other_ddG)
        elif strategy == 'pam':
            for i in range(target_len-3, target_len):
                for mm_base in bases.replace(ref_target[i], ''):
                    if i == 1 and mm_base == 'C':
                        # Unreliable in two runs
                        continue
                    ref_seq = ref_target[:i] + mm_base + ref_target[i+1:]
                    other_seq = other_target[:i] + mm_base + other_target[i+1:]

                    ref_vals.append(ddGs[ref_run][ref_seq])
                    other_vals.append(ddGs[other_run][other_seq])
        elif strategy == 'neg_control':
            ref_vals.append(ddGs[ref_run][neg_control_target_by_run[ref_run]])
            other_vals.append(ddGs[other_run][neg_control_target_by_run[other_run]])
            
        def div_sq_err(dvsr):
            return sum((rval - (float(oval)/dvsr))**2 for rval, oval in zip(ref_vals, other_vals))

        res = scipy.optimize.minimize(div_sq_err, 1.0, method='Powell')
        divisor[other_run] = float(res.x)
    divisor[ref_run] = 1.0
    return divisor

In [ ]:
divisor = find_calibration_divisors(strategy=TA_calibration_strategy)
for run in runs:
    print '{}, Target {}: {:.2f}'.format(run.date, run.target_name, 1.0/divisor[run])

In [ ]:
if ABA_zero_strategy == 'neg_control':
    ABA_zero = {run: ABAs[run][neg_control_target_by_run[run]] for run in runs}
elif ABA_zero_strategy == 'max_concentration':
    ABA_zero = {run: np.log(Kds[run][neg_control_target_by_run[run]]) - np.log(max_concentration_by_run[run]) for run in runs}
else:
    assert False, 'Invalid ABA_zero_strategy'
    
for run in runs:
    ABAs[run] = {seq: (ABA - ABA_zero[run])/divisor[run] for seq, ABA in ABAs[run].items()}
    ABA_error[run] = {seq: err/divisor[run] for seq, err in ABA_error[run].items()}
    ddGs[run] = {seq: ddG/divisor[run] for seq, ddG in ddGs[run].items()}
    ddG_error[run] = {seq: err/divisor[run] for seq, err in ddG_error[run].items()}

## Model

In [ ]:
total_seqs = 0
rows, cols, tmp_data = [], [], []
for run in runs:
    print run
    target = targets[run.target_name]
    for i, seq in enumerate(single_ham_seqs(target)):
        tmp_data.append((ddGs[run][seq]))
        v = sequence_transition_vector(target, seq)
        for j, val in enumerate(v):
            if val:
                rows.append(total_seqs)
                cols.append(j)
        total_seqs += 1
A = scipy.sparse.coo_matrix((np.ones((len(rows),)), (rows, cols)), 
                            shape=(total_seqs, target_len*12), dtype=np.int)
A = A.tocsr()
data = np.array(tmp_data)

In [ ]:
def score_free_transition(params):
    return np.linalg.norm(A.dot(np.outer(params[:target_len], params[target_len:]).flatten()) - data)

x0 = np.ones((target_len+12,))
for i in range(4):
    for j in range(3):
        if j < i:
            val = M[ref_run][i, j]
        else:
            val = M[ref_run][i, j+1]
    x0[target_len + 3*i + j] = val
res = scipy.optimize.minimize(score_free_transition, x0, method='Powell')
print 'Success:', res.success

In [ ]:
pred = A.dot(np.outer(res.x[:target_len], res.x[target_len:]).flatten())

In [ ]:
from scipy.stats import pearsonr

In [ ]:
fig = plt.figure(figsize=(15, 4))

# Transition weights
ax = plt.subplot2grid((1, 3), (0, 1))
Mpred = np.zeros((4, 4))
for i in range(4):
    for j in range(3):
        val = res.x[target_len + 3*i + j]
        if j < i:
            Mpred[i, j] = val
        else:
            Mpred[i, j+1] = val
sum_M = Mpred.sum()
vals = [v for v in Mpred.flatten() if v > 0]
avg_M = np.average(vals)
Mpred /= avg_M
extremum = max(abs(v-1) for v in Mpred.flatten() if v > 0)
for i in range(4):
    Mpred[i, i] = None


ms = ax.matshow(Mpred, cmap=plt.get_cmap('coolwarm'), vmin=1-extremum, vmax=1+extremum)
ax.grid(False)
ax.set_ylabel('From')
ax.set_xlabel('To')
ax.set_xticks(range(4))
ax.set_yticks(range(4))
ax.set_xticklabels(bases)
ax.set_yticklabels(bases)
ax.xaxis.set_ticks_position('bottom')
ax.set_title('Transition Weights')
#ax.set_axis_bgcolor('white')

cbar = plt.colorbar(ms)
cbar.set_label(r'Weight', fontsize=10)


# Position Weights
ax = plt.subplot2grid((1, 3), (0, 0))
pos_weights = avg_M * res.x[:target_len]
ax.plot(range(target_len), pos_weights)
xlim = (0, target_len - 1)
ax.plot(xlim, [0]*2, 'k:', zorder=-1)
ax.set_xlabel('Position')
ax.set_ylabel('Penalty ($k_B T$)')
ax.set_title('Position Penalties')
#ticks = [-2, 0, 1] + range(6, target_len - 2, 6)
#labels = [-3, -1] + ticks[2:]
#ax.set_xticks(ticks)
#ax.set_xticklabels(labels)
ax.set_xticks(range(0, target_len, 5))
ax.set_xlim(xlim)
ax.set_axis_bgcolor('white')
ax.grid(False)

# Predicted v Measures
ax = plt.subplot2grid((1, 3), (0, 2))
ddG_vals = []
for run in runs:
    ddG_vals.extend([(ddGs[run][seq])
                     for seq in single_ham_seqs(targets[run.target_name])])

for i, run in enumerate(runs):
    start = target_len*3 * i
    end = target_len*3 * (i+1)
    ax.plot(ddG_vals[start:end], pred[start:end], '.',
            label='T{}, {}-{}'.format(run.target_name, run.date[-4:-2], run.date[-2:]))
ax.set_xlabel(r'Measured $\Delta \Delta G$ ($k_B T$)')
ax.set_ylabel(r'Fit $\Delta \Delta G$ ($k_B T$)')

r, pval = pearsonr(ddG_vals, pred)
ax.text(0.25, 1, '$r = %.2f$' % r, fontsize=16, va='center', ha='center')
ax.set_title('Fit vs. Measured $\Delta \Delta G$')
ax.set_aspect(1)
ax.set_xticks(range(0, 2))
ax.set_yticks(range(0, 2))
ax.legend(loc='lower right')
ax.get_legend().get_frame().set_facecolor('white')
ax.set_axis_bgcolor('white')
ax.grid(False)

xlim, ylim = ax.get_xlim(), ax.get_ylim()
min_low = min(xlim[0], ylim[0])
max_high = max(xlim[1], ylim[1])
lim = (min_low, max_high)
ax.plot(lim, lim, 'k', alpha=0.5, zorder=-1)
ax.set_xlim(lim)
ax.set_ylim(lim)

fig.savefig(os.path.join(fig_dir, 'TA_TC_position_transition_model_all.png'), dpi=300)
fig.savefig(os.path.join(fig_dir, 'TA_TC_position_transition_model_all.eps'))
print

In [ ]:
with open(os.path.join(results_dir, 'position_transition_weights.txt'), 'w') as out:
    out.write('Transition weights:\n')
    for row, from_base in zip(Mpred, bases):
        out.write('\t'.join([from_base] + ['%.3f' % w for w in row]) + '\n')
    out.write('\t'.join(['From/To'] + list(bases)) + '\n')
    out.write('\n')
    out.write('Position weights:\n')
    out.write('\n'.join(['%.3f' % w for w in pos_weights]))

# All ABAs

In [ ]:
from scipy.interpolate import interp1d

In [ ]:
run = runs[0]
all_ABAs = [(ABA, seq) for seq, ABA in ABAs[run].items()]
all_ABAs.sort(reverse=True)

fig, ax = plt.subplots(figsize=(8, 7))
x = np.array([i for i, (ABA, seq) in enumerate(all_ABAs) if ABA > 0])
print 'Num points:', len(x)
y = np.array([ABA for i, (ABA, seq) in enumerate(all_ABAs) if ABA > 0])
yerr = np.array([ABA_error[run][seq] for i, (ABA, seq) in enumerate(all_ABAs) if ABA > 0])
ylb = y - yerr
yub = y + yerr
err_color = 0.9*np.ones((3,))
ax.fill_between(list(x), list(ylb), list(yub), facecolor=err_color, alpha=1.0, color=err_color)
sc = ax.scatter(x, y, c='firebrick', s=30, linewidth=0)

ax.set_axis_bgcolor('white')
ax.grid(False)

ax.set_xlabel('Rank', fontsize=18)
ax.set_ylabel('Apparent Binding Affinity ($k_B T$)', fontsize=18)
ax.set_title('All Apparent Binding Affinities by Rank', fontsize=20)

#ax.set_xticks(range(0, len(x), 2000))
#ax.set_yticks(range(0, 8, 2))
ax.set_xlim((-len(x)*0.01, len(x)*1.01))
#ax.set_ylim((-0.5, 7))
for item in ax.get_xticklabels() + ax.get_yticklabels():
    item.set_fontsize(16)
    
fig.savefig(os.path.join(fig_dir, 'ranked_affinities.png'), dpi=300)
fig.savefig(os.path.join(fig_dir, 'ranked_affinities.eps'))

# Reproducibility

In [ ]:
from champ.adapters_cython import simple_hamming_distance

In [ ]:
def first_mismatch(s1, s2):
    for i, (c1, c2) in enumerate(zip(s1, s2)):
        if c1 != c2:
            return i
    return -1

In [ ]:
def discrete_cmap(N, base_cmap=None):
    """Create an N-bin discrete colormap from the specified input map"""

    # Note that if base_cmap is a string or None, you can simply do
    #    return plt.cm.get_cmap(base_cmap, N)
    # The following works for string, None, or a colormap instance:

    base = plt.get_cmap(base_cmap)
    color_list = base(np.linspace(0, 1, N))
    cmap_name = base.name + str(N)
    return base.from_list(cmap_name, color_list, N)

In [ ]:
print 'Max Hamming Dist:'
for run in runs:
    print run.date, max(simple_hamming_distance(target_by_run[run], seq) for seq in ddGs[run])

In [ ]:
runX = next(run for run in runs if '20160910' in run.date)
runY = next(run for run in runs if '20160911' in run.date)

thresh = 7

def flipout_mm(target, seq):
    for idx in flipout_idxs:
        if target[idx] != seq[idx]:
            return True
    return False

seqs = [seq for seq in ABAs[runX].keys() if seq in ABAs[runY]]
seqs.sort(key=lambda s: ABAs[runX][s])
seqs_and_Xidx = [(seq, i) for i, seq in enumerate(seqs)]
seqs_and_Xidx.sort(key=lambda tup: ABAs[runY][tup[0]])
x = [idx for seq, idx in seqs_and_Xidx]
y = range(len(seqs))

has_flipout_mm = np.array([flipout_mm(target_by_run[runY], seq) for seq, _ in seqs_and_Xidx])
    
fig, ax = plt.subplots(figsize=(7, 9))
ax.scatter(x, y, c='b', s=20*has_flipout_mm, linewidth=0, label='Has Flipout Substitution')
ax.scatter(x, y, c='firebrick', s=20*(1-has_flipout_mm), linewidth=0, label='No Flipout Substitution')
ax.set_aspect(1)
legend = ax.legend(loc='upper left')
legend.get_frame().set_facecolor('white')

ax.set_xlabel('ABA Rank, {r.date}, Chip {r.project_name}'.format(r=runX), fontsize=18)
ax.set_ylabel('ABA Rank, {r.date}, Chip {r.project_name}'.format(r=runY), fontsize=18)
ax.set_title('Comparison of ABA between Runs', fontsize=20)

ax.set_axis_bgcolor('white')
ax.grid(False)
for item in ax.get_xticklabels() + ax.get_yticklabels():
    item.set_fontsize(16)
    
ax.set_xlim((0, len(seqs)+100))
ax.set_ylim((0, len(seqs)+100))

r, pval = pearsonr(x, y)
ax.text(1500, 3300, r'$\rho = {:.2f}$'.format(r), fontsize=18, ha='center', va='center')

#fig.savefig(os.path.join(fig_dir, 'aba_0121_v_0714_rank.png'), dpi=300)
#fig.savefig(os.path.join(fig_dir, 'aba_0121_v_0714_rank.eps'))

# Dinucleotide Data

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
from champ.adapters_cython import simple_hamming_distance
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl

In [ ]:
color_given_frac = {
    0.0: np.array([1, 1, 1], dtype=np.float),
    0.5: np.array([0.9, 0.4, 0.2], dtype=np.float),
    0.8: np.array([0.9, 0, 0], dtype=np.float),
    1.0: np.array([0.5, 0, 0], dtype=np.float)
}
color_frac_list = list(sorted(color_given_frac.keys()))
color_list = [color_given_frac[frac] for frac in color_frac_list]

mycmap = mpl.colors.LinearSegmentedColormap.from_list('mycmap', zip(color_frac_list, color_list))

def color_by_frac(frac):
    for f1, f2 in zip(color_frac_list, color_frac_list[1:]):
        if f1 <= frac <= f2:
            c1 = color_given_frac[f1]
            c2 = color_given_frac[f2]
            return c1 + (frac-f1)/(f2-f1) * (c2 - c1)
    raise ValueError('Invalid frac entered: %.3g' % frac)

In [ ]:
def label_matrix(ax, target):
    patch_offset = 1.0
    total_w = 0.8 * 3
    patch_w = total_w / 3.0
    patch_h = 1.0
    
    x_text_offset = patch_offset + patch_h + 1.5
    y_text_offset = x_text_offset + 0.5
    fontsize = 12
    
    for i, ref_base in enumerate(target):
        base_pos = 3*i + 1.3
        ax.text(-y_text_offset, base_pos, ref_base, 
                color=color_given_base[ref_base],
                ha='center', va='center', fontweight='bold')
        ax.text(base_pos, -x_text_offset, ref_base, 
                color=color_given_base[ref_base],
                ha='center', va='center', fontweight='bold')
        
        offsets = [-total_w/2.0, -patch_w/2.0, patch_w/2.0]
        for new_base, offset in zip(bases.replace(ref_base, ''), offsets):
            pos = base_pos + offset
            ax.add_patch(
                patches.Rectangle((pos, -patch_offset), patch_w, -patch_h,
                                  fill=True, facecolor=color_given_base[new_base], 
                                  linewidth=0, clip_on=False)
            )
            ax.add_patch(
                patches.Rectangle((-patch_offset, pos), -patch_h, patch_w,
                                  fill=True, facecolor=color_given_base[new_base], 
                                  linewidth=0, clip_on=False)
            )

In [ ]:
def lower_label_matrix(ax, target, maxy):
    maxy -= 1
    
    patch_offset = 1.0
    total_w = 0.8 * 3
    patch_w = total_w / 3.0
    patch_h = 1.0
    
    x_text_offset = patch_offset + patch_h + 1.5
    y_text_offset = x_text_offset + 0.5
    fontsize = 12
    
    for i, ref_base in enumerate(target):
        base_pos = 3*i + 1.3
        ax.text(-y_text_offset, base_pos, ref_base, 
                color=color_given_base[ref_base],
                ha='center', va='center', fontweight='bold')
        ax.text(base_pos, maxy + x_text_offset, ref_base, 
                color=color_given_base[ref_base],
                ha='center', va='center', fontweight='bold')
        
        offsets = [-total_w/2.0, -patch_w/2.0, patch_w/2.0]
        for new_base, offset in zip(bases.replace(ref_base, ''), offsets):
            pos = base_pos + offset
            ax.add_patch(
                patches.Rectangle((pos, maxy + patch_offset), patch_w, patch_h,
                                  fill=True, facecolor=color_given_base[new_base], 
                                  linewidth=0, clip_on=False)
            )
            ax.add_patch(
                patches.Rectangle((-patch_offset, pos), -patch_h, patch_w,
                                  fill=True, facecolor=color_given_base[new_base], 
                                  linewidth=0, clip_on=False)
            )

In [ ]:
fig, ax = plt.subplots(figsize=(11, 9))

loc_targets = {run: targets[run.target_name] for run in runs}
loc_runs = [runs[0]]
slen = target_len * 3
D = {run: np.zeros((slen, slen)) for run in loc_runs}
print(loc_runs)
for run_idx, run in enumerate(loc_runs):
    target = loc_targets[run]
    D[run][:, :] = None
    for i in range(len(target)):
        for j in range(i):
            for k, b1 in enumerate(bases.replace(target[j], '')):
                for l, b2 in enumerate(bases.replace(target[i], '')):
                    seq = target[:j] + b1 + target[j+1:i] + b2 + target[i+1:]
                    if run_idx == 0:
                        r, c = i*3+l, j*3+k
                    else:
                        r, c = j*3+k, i*3+l
                    if seq in ABAs[run]:
                        if ABAs[run][seq] > 0:
                            D[run][r, c] = ABAs[run][seq]
                        else:
                            D[run][r, c] = 0
    for i in range(len(target)):
        for l, b1 in enumerate(bases.replace(target[j], '')):
            seq = target[:i] + b1 + target[i+1:]
            r = i*3 + l
            if seq in ABAs[run]:
                if ABAs[run][seq] > 0:
                    D[run][r, r] = ABAs[run][seq]
                else:
                    D[run][r, r] = 0

Both = np.zeros((slen, slen))
Both[:, :] = None
for i in range(slen):
    for j in range(i - i%3):
        Both[i, j] = D[loc_runs[0]][i, j]
        Both[j, i] = 0
    Both[i, i] = D[loc_runs[0]][i, i]
ms = ax.matshow(Both, cmap=mycmap) #plt.get_cmap('Reds'))

xlim = ax.get_xlim()
ylim = ax.get_ylim()
for i in np.arange(2.5, slen, 3):
    ax.plot(xlim, [i, i], 'w', alpha=1, linewidth=1)
    ax.plot([i, i], ylim, 'w', alpha=1, linewidth=1)
ax.grid(False)

lower_label_matrix(ax, target, Both.shape[0])

ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.set_xticks([])
ax.set_yticks([])

ax.set_axis_bgcolor(0.87*np.array([1, 1, 1]))

fs = 18
ax.set_title(r'AsCpf1 2016-10-17 Dinucleotide ABA', fontsize=fs)

cbar = plt.colorbar(ms)
cbar.set_label(r'ABA ($k_B T$)', fontsize=fs)

fig.savefig(os.path.join(fig_dir, 'aba_matrix_avg.png'), dpi=300)
fig.savefig(os.path.join(fig_dir, 'aba_matrix_avg.eps'))

In [ ]:
fig, ax = plt.subplots(figsize=(11, 9))

loc_runs = [runs[0]]
print(targets)
loc_targets = {run: targets[run.target_name] for run in loc_runs}

slen = target_len * 3
D = {run: np.zeros((slen, slen)) for run in loc_runs}
for run_idx, run in enumerate(loc_runs):
    target = loc_targets[run]
    D[run][:, :] = None
    for i in range(len(target)):
        for j in range(i):
            for k, b1 in enumerate(bases.replace(target[j], '')):
                for l, b2 in enumerate(bases.replace(target[i], '')):
                    seq = target[:j] + b1 + target[j+1:i] + b2 + target[i+1:]
                    sin_mm_seq1 = target[:j] + b1 + target[j+1:]
                    sin_mm_seq2 = target[:i] + b2 + target[i+1:]
                    if run_idx == 0:
                        r, c = i*3+l, j*3+k
                    else:
                        r, c = j*3+k, i*3+l
                    if seq in ABAs[run] and sin_mm_seq1 in ABAs[run] and sin_mm_seq2 in ABAs[run]:
                        D[run][r, c] = (ddGs[run][seq] - ddGs[run][sin_mm_seq1] - ddGs[run][sin_mm_seq2])

Both = np.zeros((slen, slen))
Both[:, :] = None
for i in range(slen):
    for j in range(i - i%3):
        Both[i, j] = D[loc_runs[0]][i, j]
        Both[j, i] = 0
    for j in range(i - i%3, (i+3) - i%3):
        Both[i, j] = 0
        Both[j, i] = 0

extreme = max(
    abs(min(v for v in Both.flatten() if not np.isnan(v))),
    abs(min(v for v in Both.flatten() if not np.isnan(v)))
)

ms = ax.matshow(Both, cmap=plt.get_cmap('seismic'), vmin=-extreme, vmax=extreme)

xlim = ax.get_xlim()
ylim = ax.get_ylim()
for i in np.arange(2.5, slen, 3):
    ax.plot(xlim, [i, i], 'w', alpha=1, linewidth=1)
    ax.plot([i, i], ylim, 'w', alpha=1, linewidth=1)
ax.grid(False)

lower_label_matrix(ax, target, Both.shape[0])

ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.set_xticks([])
ax.set_yticks([])

fs = 18
ax.set_title(r'AsCpf1 2016-10-17 Dinucleotide $\Delta \Delta G$ "Epistasis"', fontsize=fs)

cbar = plt.colorbar(ms)
cbar.set_label(r'$\Delta \Delta G_{1, 2} - (\Delta \Delta G_1 + \Delta \Delta G_2) \ \left(k_B T\right)$', fontsize=fs)

fig.savefig(os.path.join(fig_dir, 'epistasis_matrix_avg.png'), dpi=300)
fig.savefig(os.path.join(fig_dir, 'epistasis_matrix_avg.eps'))

In [ ]:
fig, ax = plt.subplots(figsize=(11, 9))

loc_runs = runs[:2]
loc_targets = {run: targets[run.target_name] for run in loc_runs}

slen = target_len * 3
D = {run: np.zeros((slen, slen)) for run in loc_runs}
for run_idx, run in enumerate(loc_runs):
    target = loc_targets[run]
    D[run][:, :] = None
    for i in range(len(target)):
        for j in range(i):
            for k, b1 in enumerate(bases.replace(target[j], '')):
                for l, b2 in enumerate(bases.replace(target[i], '')):
                    seq = target[:j] + b1 + target[j+1:i] + b2 + target[i+1:]
                    if run_idx == 0:
                        r, c = i*3+l, j*3+k
                    else:
                        r, c = j*3+k, i*3+l
                    if seq in ABAs[run]:
                        if ABAs[run][seq] > 0:
                            D[run][r, c] = ABAs[run][seq]
                        else:
                            D[run][r, c] = 0

Both = np.zeros((slen, slen))
Both[:, :] = None
for i in range(slen):
    for j in range(i - i%3):
        Both[i, j] = D[loc_runs[0]][i, j]
        Both[j, i] = D[loc_runs[1]][j, i]
ms = ax.matshow(Both, cmap=plt.get_cmap('Reds'))

xlim = ax.get_xlim()
ylim = ax.get_ylim()
for i in np.arange(2.5, slen, 3):
    ax.plot(xlim, [i, i], 'w', alpha=1, linewidth=1)
    ax.plot([i, i], ylim, 'w', alpha=1, linewidth=1)
ax.grid(False)

label_matrix(ax, target)

ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.set_xticks([])
ax.set_yticks([])

ax.set_axis_bgcolor(0.87*np.array([1, 1, 1]))

fs = 18
ax.set_title(r'Dinucleotide ABA' + '\nLower: {}, Upper {}'.format(loc_runs[0].date, loc_runs[1].date), fontsize=fs)

cbar = plt.colorbar(ms)
cbar.set_label(r'ABA ($k_B T$)', fontsize=fs)

fig.savefig(os.path.join(fig_dir, 'aba_matrix.png'), dpi=300)
fig.savefig(os.path.join(fig_dir, 'aba_matrix.eps'))

In [ ]:
fig, ax = plt.subplots(figsize=(11, 9))

loc_runs = runs[::2]
loc_targets = {run: targets[run.target_name] for run in loc_runs}

slen = target_len * 3
D = {run: np.zeros((slen, slen)) for run in loc_runs}
for run_idx, run in enumerate(loc_runs):
    target = loc_targets[run]
    D[run][:, :] = None
    for i in range(len(target)):
        for j in range(i):
            for k, b1 in enumerate(bases.replace(target[j], '')):
                for l, b2 in enumerate(bases.replace(target[i], '')):
                    seq = target[:j] + b1 + target[j+1:i] + b2 + target[i+1:]
                    if run_idx == 0:
                        r, c = i*3+l, j*3+k
                    else:
                        r, c = j*3+k, i*3+l
                    if seq in ABAs[run]:
                        if ABAs[run][seq] > 0:
                            D[run][r, c] = ABAs[run][seq]
                        else:
                            D[run][r, c] = 0

Both = np.zeros((slen, slen))
Both[:, :] = None
for i in range(slen):
    for j in range(i - i%3):
        Both[i, j] = D[loc_runs[0]][i, j]
        Both[j, i] = D[loc_runs[1]][j, i]
ms = ax.matshow(Both, cmap=plt.get_cmap('Reds'))

xlim = ax.get_xlim()
ylim = ax.get_ylim()
for i in np.arange(2.5, slen, 3):
    ax.plot(xlim, [i, i], 'w', alpha=1, linewidth=1)
    ax.plot([i, i], ylim, 'w', alpha=1, linewidth=1)
ax.grid(False)

label_matrix(ax, target)

ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.set_xticks([])
ax.set_yticks([])

ax.set_axis_bgcolor(0.87*np.array([1, 1, 1]))

fs = 18
ax.set_title(r'Dinucleotide ABA' + '\nLower: {}, Upper {}'.format(loc_runs[0].date, loc_runs[1].date), fontsize=fs)

cbar = plt.colorbar(ms)
cbar.set_label(r'ABA ($k_B T$)', fontsize=fs)

fig.savefig(os.path.join(fig_dir, 'aba_matrix.png'), dpi=300)
fig.savefig(os.path.join(fig_dir, 'aba_matrix.eps'))

In [ ]:
fig, ax = plt.subplots(figsize=(11, 9))

loc_runs = runs[:2]
loc_targets = {run: targets[run.target_name] for run in loc_runs}

slen = target_len * 3
D = {run: np.zeros((slen, slen)) for run in loc_runs}
for run_idx, run in enumerate(loc_runs):
    target = loc_targets[run]
    D[run][:, :] = None
    for i in range(len(target)):
        for j in range(i):
            for k, b1 in enumerate(bases.replace(target[j], '')):
                for l, b2 in enumerate(bases.replace(target[i], '')):
                    seq = target[:j] + b1 + target[j+1:i] + b2 + target[i+1:]
                    sin_mm_seq1 = target[:j] + b1 + target[j+1:]
                    sin_mm_seq2 = target[:i] + b2 + target[i+1:]
                    if run_idx == 0:
                        r, c = i*3+l, j*3+k
                    else:
                        r, c = j*3+k, i*3+l
                    if seq in ABAs[run] and sin_mm_seq1 in ABAs[run] and sin_mm_seq2 in ABAs[run]:
                        D[run][r, c] = (ddGs[run][seq] - ddGs[run][sin_mm_seq1] - ddGs[run][sin_mm_seq2])

Both = np.zeros((slen, slen))
Both[:, :] = None
for i in range(slen):
    for j in range(i - i%3):
        Both[i, j] = D[loc_runs[0]][i, j]
        Both[j, i] = D[loc_runs[1]][j, i]

extreme = max(
    abs(min(v for v in Both.flatten() if not np.isnan(v))),
    abs(min(v for v in Both.flatten() if not np.isnan(v)))
)

ms = ax.matshow(Both, cmap=plt.get_cmap('seismic'), vmin=-extreme, vmax=extreme)

xlim = ax.get_xlim()
ylim = ax.get_ylim()
for i in np.arange(2.5, slen, 3):
    ax.plot(xlim, [i, i], 'w', alpha=1, linewidth=1)
    ax.plot([i, i], ylim, 'w', alpha=1, linewidth=1)
ax.grid(False)

label_matrix(ax, target)

ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.set_xticks([])
ax.set_yticks([])

fs = 18
ax.set_title(r'Dinucleotide $\Delta \Delta G$ "Epistasis"' + '\nLower: {}, Upper {}'.format(loc_runs[0].date, loc_runs[1].date), fontsize=fs)

cbar = plt.colorbar(ms)
cbar.set_label(r'$\Delta \Delta G_{1, 2} - (\Delta \Delta G_1 + \Delta \Delta G_2) \ \left(k_B T\right)$', fontsize=fs)

fig.savefig(os.path.join(fig_dir, 'epistasis_matrix.png'), dpi=300)
fig.savefig(os.path.join(fig_dir, 'epistasis_matrix.eps'))

In [ ]:
fig, ax = plt.subplots(figsize=(11, 9))

loc_runs = runs[::2]
loc_targets = {run: targets[run.target_name] for run in loc_runs}

slen = target_len * 3
D = {run: np.zeros((slen, slen)) for run in loc_runs}
for run_idx, run in enumerate(loc_runs):
    target = loc_targets[run]
    D[run][:, :] = None
    for i in range(len(target)):
        for j in range(i):
            for k, b1 in enumerate(bases.replace(target[j], '')):
                for l, b2 in enumerate(bases.replace(target[i], '')):
                    seq = target[:j] + b1 + target[j+1:i] + b2 + target[i+1:]
                    sin_mm_seq1 = target[:j] + b1 + target[j+1:]
                    sin_mm_seq2 = target[:i] + b2 + target[i+1:]
                    if run_idx == 0:
                        r, c = i*3+l, j*3+k
                    else:
                        r, c = j*3+k, i*3+l
                    if seq in ABAs[run] and sin_mm_seq1 in ABAs[run] and sin_mm_seq2 in ABAs[run]:
                        D[run][r, c] = (ddGs[run][seq] - ddGs[run][sin_mm_seq1] - ddGs[run][sin_mm_seq2])

Both = np.zeros((slen, slen))
Both[:, :] = None
for i in range(slen):
    for j in range(i - i%3):
        Both[i, j] = D[loc_runs[0]][i, j]
        Both[j, i] = D[loc_runs[1]][j, i]

extreme = max(
    abs(min(v for v in Both.flatten() if not np.isnan(v))),
    abs(min(v for v in Both.flatten() if not np.isnan(v)))
)

ms = ax.matshow(Both, cmap=plt.get_cmap('seismic'), vmin=-extreme, vmax=extreme)

xlim = ax.get_xlim()
ylim = ax.get_ylim()
for i in np.arange(2.5, slen, 3):
    ax.plot(xlim, [i, i], 'w', alpha=1, linewidth=1)
    ax.plot([i, i], ylim, 'w', alpha=1, linewidth=1)
ax.grid(False)

label_matrix(ax, target)

ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.set_xticks([])
ax.set_yticks([])

fs = 18
ax.set_title(r'Dinucleotide $\Delta \Delta G$ "Epistasis"' + '\nLower: {}, Upper {}'.format(loc_runs[0].date, loc_runs[1].date), fontsize=fs)

cbar = plt.colorbar(ms)
cbar.set_label(r'$\Delta \Delta G_{1, 2} - (\Delta \Delta G_1 + \Delta \Delta G_2) \ \left(k_B T\right)$', fontsize=fs)

fig.savefig(os.path.join(fig_dir, 'epistasis_matrix.png'), dpi=300)
fig.savefig(os.path.join(fig_dir, 'epistasis_matrix.eps'))

# 6N PAMs

## PAM ABAs

In [ ]:
run = runs[0]
pams = map(''.join, itertools.product(bases, repeat=4))
targ = target_by_run[run][4:]
print(targ)
pamseqs = [pam + targ for pam in pams]
print(len(pamseqs))
print(len(ABAs[run]))
print '6N pams found:', sum(1 for pamseq in pamseqs if pamseq in ABAs[run])
pam_ABAs, pam_ABA_error = {}, {}
for pamseq in pamseqs:
    pam = pamseq[:5]
    if pamseq in ABAs[run]:
        ABA = ABAs[run][pamseq]
        err = ABA_error[run][pamseq]
    else:
        ABA, err = 0, 0
    pam_ABAs[pam] = max(0, ABA)
    pam_ABA_error[pam] = err

## Violin Plots

In [ ]:
all_triplets = map(''.join, itertools.product(bases, repeat=3))

In [ ]:
def max_diff(pam_with_one_N, as_pct=True):
    assert pam_with_one_N.count('N') == 1, pam_with_one_N
    idx = pam_with_one_N.index('N')
    max_diff = -1
    for b1, b2 in itertools.combinations(bases, r=2):
        pam1 = pam_with_one_N[:idx] + b1 + pam_with_one_N[idx+1:]
        pam2 = pam_with_one_N[:idx] + b2 + pam_with_one_N[idx+1:]
        if pam_ABAs[pam1] > 0 and pam_ABAs[pam2] > 0:
            diff = abs(pam_ABAs[pam1] - pam_ABAs[pam2])
            diff_stdev = np.sqrt(pam_ABA_error[pam1]**2 + pam_ABA_error[pam2]**2)
            diff -= 2.57 * diff_stdev
            if as_pct:
                diff /= max(pam_ABAs[pam1], pam_ABAs[pam2])
            if diff > max_diff:
                max_diff = diff
    if max_diff == -1:
        return False
    else:
        return max(0, max_diff)

In [ ]:
def max_diff_seqs(pam_with_one_N):
    assert pam_with_one_N.count('N') == 1, pam_with_one_N
    idx = pam_with_one_N.index('N')
    max_diff = -1
    max_pams = []
    for b1, b2 in itertools.combinations(bases, r=2):
        pam1 = pam_with_one_N[:idx] + b1 + pam_with_one_N[idx+1:]
        pam2 = pam_with_one_N[:idx] + b2 + pam_with_one_N[idx+1:]
        if pam_ABAs[pam1] > 0 and pam_ABAs[pam2] > 0:
            diff = abs(pam_ABAs[pam1] - pam_ABAs[pam2])
            if diff > max_diff: #diff > 0 and diff > max_diff:
                max_diff = diff
                max_pams = (pam1, pam2)
    if max_diff == -1:
        return False
    else:
        return max_pams

In [ ]:
as_pct = True
max_diffs = {i: [] for i in range(6)}
all_quintuplets = map(''.join, itertools.product(bases, repeat=5))
for i in range(6):
    for quint in all_quintuplets:
        pam_with_one_N = quint[:i] + 'N' + quint[i:]
        diff = max_diff(pam_with_one_N, as_pct)
        if diff is not False:
            max_diffs[i].append(diff)
        if i <= 2 and diff > 0.3:
            pam1, pam2 = max_diff_seqs(pam_with_one_N)
            ABA1, ABA2 = pam_ABAs[pam1], pam_ABAs[pam2]
#            print '{}, {}: {}={}, {}={} ({})'.format(i, diff, pam1, ABA1, pam2, ABA2,
#                                                     max(ABA1, ABA2)/min(ABA1, ABA2))

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
for range_lim, ax in zip([3, 6], axes):
    viol_d = ax.violinplot([max_diffs[i] for i in range(range_lim)], 
                           positions=range(range_lim), 
                           showextrema=False)
    for bod in viol_d['bodies']:
        bod.set_color('grey')
        bod.set_alpha(1)
        bod.set_edgecolor('k')
    ax.grid(False)
    ax.set_axis_bgcolor('white')
    ax.set_xticks(range(range_lim))
    ax.set_xticklabels(range(-6, -6 + range_lim))
    #ax.set_yticks(range(5))
    ax.set_xlabel('Single Substitution Position', fontsize=16)
    ax.set_ylabel('Max Pct ABA Reduction\n95% Confidence Lower Bound', fontsize=16)
    
    ylim = ax.get_ylim()
    ub = ylim[1] * 1.2
    for i in range(range_lim):
        ax.text(i, ylim[1] * 1.1, str(len(max_diffs[i])), ha='center', va='center', fontsize=12)
    ax.set_ylim((ylim[0], ub))
fig.savefig(os.path.join(fig_dir, 'pam_6N_violins_pct.eps'))
fig.savefig(os.path.join(fig_dir, 'pam_6N_violins_pct.png'), dpi=300)

In [ ]:
as_pct = False
max_diffs = {i: [] for i in range(6)}
all_quintuplets = map(''.join, itertools.product(bases, repeat=5))
for i in range(6):
    for quint in all_quintuplets:
        pam_with_one_N = quint[:i] + 'N' + quint[i:]
        diff = max_diff(pam_with_one_N, as_pct)
        if diff is not False:
            max_diffs[i].append(diff)
        if i <= 2 and diff > 0.3:
            pam1, pam2 = max_diff_seqs(pam_with_one_N)
            ABA1, ABA2 = pam_ABAs[pam1], pam_ABAs[pam2]
#            print '{}, {}: {}={}, {}={} ({})'.format(i, diff, pam1, ABA1, pam2, ABA2,
#                                                     max(ABA1, ABA2)/min(ABA1, ABA2))

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
for range_lim, ax in zip([3, 6], axes):
    viol_d = ax.violinplot([max_diffs[i] for i in range(range_lim)], 
                           positions=range(range_lim), 
                           showextrema=False)
    for bod in viol_d['bodies']:
        bod.set_color('grey')
        bod.set_alpha(1)
        bod.set_edgecolor('k')
    ax.grid(False)
    ax.set_axis_bgcolor('white')
    ax.set_xticks(range(range_lim))
    ax.set_xticklabels(range(-6, -6 + range_lim))
    #ax.set_yticks(range(5))
    ax.set_xlabel('Single Substitution Position', fontsize=16)
    ax.set_ylabel('Max $\Delta$ABA ($k_b T$)\n95% Confidence Lower Bound', fontsize=16)
    
    ylim = ax.get_ylim()
    ub = ylim[1] * 1.2
    for i in range(range_lim):
        ax.text(i, ylim[1] * 1.1, str(len(max_diffs[i])), ha='center', va='center', fontsize=12)
    ax.set_ylim((ylim[0], ub))
fig.savefig(os.path.join(fig_dir, 'pam_6N_violins.eps'))
fig.savefig(os.path.join(fig_dir, 'pam_6N_violins.png'), dpi=300)

In [ ]:
print 'PAMs considered:', len(pams)
print 'PAMs with ABA > 0:', sum(1 for pam in pams if pam_ABAs[pam] > 0)

## PAM Wheel Data

In [ ]:
pams.sort(key=lambda x: x[::-1])  # sort from last base to first base

In [ ]:
out_fpath = os.path.join(results_dir, 'pam_wheel_ABA_data.txt')
with open(out_fpath, 'w') as out:
    for pam in pams:
        out.write('\t'.join([str(pam_ABAs[pam])] + list(pam)) + '\n')

## PAM Landscape

In [ ]:
def sorted_pams_at_ham_dist_by_base_pam_and_Kd(i):
    all_triplets = map(''.join, itertools.product(bases, repeat=3))
    ham_trips = [trip for trip in all_triplets
                 if simple_hamming_distance(trip, local_config.targets['A'][:3]) == i]
    ham_pams = []
    for base_pam in ham_trips:
        base_pam_pams = [trip + base_pam for trip in all_triplets]
        base_pam_pams.sort(key=lambda p: -pam_ABAs[p])
        ham_pams.append(base_pam_pams)
    return ham_pams

In [ ]:
mx_height = max(pam_ABAs[pam] for pam in pams)

w = 0.9
text_off = 0.15
alpha=0.7
fig, ax = plt.subplots(figsize=(14, 10))
base_rs = [0.25, 1.5, 2.5]
for i in range(3):
    ham_pam_groups = sorted_pams_at_ham_dist_by_base_pam_and_Kd(i)
    assert len(ham_pam_groups) >= 1, i
    if i == 0:
        start_angle = 60 # degrees
        sa_rad = np.pi/180.0 * start_angle
        base_thetas = np.linspace(np.pi - sa_rad, sa_rad, len(ham_pam_groups) + 1)
    else:
        base_thetas = np.linspace(np.pi, -np.pi, len(ham_pam_groups) + 1)
    circ_xs = []
    circ_ys = []
    for j, ham_pams in enumerate(ham_pam_groups):
        th0, th1 = base_thetas[j:j+2]
        thetas = (th1 - th0) / float(len(ham_pams)) * np.arange(len(ham_pams)) + th0
        heights = [pam_ABAs[pam] for pam in ham_pams]
        r = base_rs[i]
        
        for theta, height, pam in zip(thetas, heights, ham_pams):
            # Data values
            scaled_height = w*(height/mx_height)
            scaled_error = w*(pam_ABA_error[pam]/mx_height)
            rs = np.array([r, r + scaled_height])
            x = np.cos(theta) * rs
            y = np.sin(theta) * rs
            #color = c1 + height/mx_height * (c2 - c1)
            ax.plot(x, y, color=color_by_frac(height/mx_height), linewidth=2)
            circ_xs.append(x[0])
            circ_ys.append(y[0])
            
            if scaled_height > 0:
                # Errors
                rs = np.array([r + scaled_height, r + scaled_height + scaled_error])
                x = np.cos(theta) * rs
                y = np.sin(theta) * rs
                ax.plot(x, y, color=0.75*np.array([1, 1, 1]), zorder=-1)
         
        pam_fs = 16
        if i == 0:
            # Add AAG label 
            ax.text(0, 0, 'AAG', ha='center', va='center', alpha=alpha, fontsize=pam_fs)
        else:
            # Add pam label
            base_pams = set(pam[3:] for pam in ham_pams)
            assert len(base_pams) == 1, base_pams
            base_pam = list(base_pams)[0]
            th_mid = (th0 + th1) / 2.0
            rr = r - text_off
            x = np.cos(th_mid) * rr
            y = np.sin(th_mid) * rr
            ax.text(x, y, base_pam, rotation=(180/np.pi*th_mid - 90), 
                    ha='center', va='center', alpha=alpha, fontsize=pam_fs)

            # Add boundary ticks
            for theta in [th0, th1]:
                rs = np.array([r - text_off, r])
                x = np.cos(theta) * rs
                y = np.sin(theta) * rs
                ax.plot(x, y, 'k', linewidth=0.3, alpha=alpha)
            
        # Add Zero Circle
        ax.plot(circ_xs, circ_ys, 'k', linewidth=0.3, alpha=alpha)
              
mx = base_rs[-1] + w/2.0
lim = (-mx, mx)
ax.set_xlim(lim)
ax.set_ylim(lim)
ax.set_aspect(1)
ax.set_xticks([])
ax.set_yticks([])
ax.set_axis_bgcolor('white')

# Color bar
#ax1 = fig.add_axes([0.95, 0.05, 0.05, 0.9])
cax, kw = mpl.colorbar.make_axes(ax)
norm = mpl.colors.Normalize(vmin=0, vmax=mx_height)
cb1 = mpl.colorbar.ColorbarBase(cax, cmap=mycmap,
                                norm=norm,
                                orientation='vertical')
cb1.set_label('ABA ($k_B T$)', fontsize=18)


ax.set_xlabel('Concentric Rings by Hamming Distance from AAG', fontsize=18)
ax.set_title('Energy Landscape of 6-Nucleotide PAMs', fontsize=20)
fig.savefig(os.path.join(fig_dir, 'energy_landscape_sawblade_maxham2.png'), dpi=300)
fig.savefig(os.path.join(fig_dir, 'energy_landscape_sawblade_maxham2.eps'))

Correlation with Huo et al Interference Efficiency
===

In [ ]:
import matplotlib as mpl
from scipy.stats import pearsonr
from scipy.optimize import curve_fit

In [ ]:
Image(filename=os.path.join(local_config.fig_dir, 'outside', 'huo_pam_efficiency.png'))

In [ ]:
huo_dir = os.path.join(local_config.data_dir, 'huo')
huo_pam_fpath = os.path.join(huo_dir, 'huo_pams.txt')
huo_csv_fpath = os.path.join(huo_dir, 'huo_data.csv')

huo_pams = ['AAA' + line.strip() for line in open(huo_pam_fpath)]
huo_efficiencies = []
huo_sds = []
with open(huo_csv_fpath) as f:
    while True:
        try:
            line = next(f)
        except StopIteration:
            break
        eff = float(line.strip().split(',')[1])
        line = next(f)
        sd = float(line.strip().split(',')[1]) - eff
        huo_efficiencies.append(eff)
        huo_sds.append(sd)
assert len(huo_pams) == len(huo_efficiencies) == len(huo_sds)

In [ ]:
reduced_huo_efficiencies, reduced_huo_sds = [], []
our_huo_ABAs, our_huo_ABA_errors = [], []
reduced_pams = []
for pam, eff, sd in zip(huo_pams, huo_efficiencies, huo_sds):
    if pam_ABAs[pam] > 0:
        reduced_huo_efficiencies.append(eff)
        reduced_huo_sds.append(sd)
        our_huo_ABAs.append(pam_ABAs[pam])
        our_huo_ABA_errors.append(pam_ABA_error[pam])
        reduced_pams.append(pam)

In [ ]:
def log_log_line(x, m, b):
    return 10**(m * np.log10(x) + b)

def log_line(x, m, b):
    return (m * np.log10(x) + b)

def exp10_line(x, m, b):
    return 10**(m*x + b)

def line(x, m, b):
    return m * x + b

In [ ]:
fit_sigmas = [np.log(eff + sd) - np.log(sd) for eff, sd in zip(reduced_huo_efficiencies, reduced_huo_sds)]

In [ ]:
popt, pcov = curve_fit(line, our_huo_ABAs, np.log10(reduced_huo_efficiencies), sigma=fit_sigmas)
r, pval = pearsonr(np.log10(reduced_huo_efficiencies), our_huo_ABAs)

xx = np.linspace(0, 6, 10)
yy = exp10_line(xx, *popt)

fig, ax = plt.subplots(figsize=(8, 7))
ax.plot(xx, yy, '--', label='Fit', linewidth=2, zorder=-1)
ax.errorbar(our_huo_ABAs, reduced_huo_efficiencies, yerr=reduced_huo_sds, xerr=our_huo_ABA_errors, 
            linestyle='.', linewidth=2, markersize=7, color='b')
ax.scatter(our_huo_ABAs, reduced_huo_efficiencies, s=46, color='b')
ax.text(1.3, 400, '$\log_{10}\, y = %.2f\, x + %.2f$\n$r = %.2f$' % (popt[0], popt[1], r), 
        fontsize=20, ha='center', va='center')
ax.set_yscale('log')
ax.set_ylim((0.5, 3000))
#ax.set_xlim((-0.8, 6.8))

fs = 20
ax.set_ylabel('Interference Efficiency', fontsize=fs, fontweight='bold')
ax.set_xlabel(r'ABA ($k_B T$)', fontsize=fs, fontweight='bold')

ax.tick_params(width=1, length=7)
ax.tick_params(width=1, length=3, which='minor')

ax.grid(False)
ax.set_axis_bgcolor('white')

for item in ax.get_xticklabels() + ax.get_yticklabels():
    item.set_fontsize(18)
    
fig.savefig(os.path.join(fig_dir, 'aba_vs_huo.eps'))
fig.savefig(os.path.join(fig_dir, 'aba_vs_huo.png'), dpi=300)

In [ ]:
print '\t'.join(['PAM', 'ABA', 'Interference Efficiency'])
for ABA, eff, pam in sorted(zip(our_huo_ABAs, reduced_huo_efficiencies, reduced_pams)):
    print pam, ABA, eff

# Write Values

In [ ]:
other_run = runs[0]
other_seqs = set(ABAs[other_run].keys())
ref_seqs = set(ABAs[ref_run].keys())
other_seqs_to_use = other_seqs - ref_seqs
print 'Ref seqs:', len(ref_seqs)
print 'Other seqs:', len(other_seqs)
print 'Intersection:', len(ref_seqs & other_seqs)
print 'Other seqs to use:', len(other_seqs_to_use)

In [ ]:
out_fname = 'Kds_and_ABAs_{}.txt'.format(dname)
out_fpath = os.path.join(results_dir, out_fname)
with open(out_fpath, 'w') as out:
    out.write('# Target: {}\n'.format(target_by_run[ref_run]))
    out.write('# Neg Control: {}\n'.format(neg_control_target_by_run[ref_run]))
    out.write('\t'.join(['# Seq', 'Kd (pM)', 'Kd error', 'ABA (kB T)', 'ABA error']) + '\n')
    out.write(
        '\n'.join(
            '\t'.join(map(str, [seq, 
                                Kds[ref_run][seq], 
                                Kd_error[ref_run][seq],
                                ABAs[ref_run][seq],
                                ABA_error[ref_run][seq]]))
            for seq in ABAs[ref_run].keys()
        )
    )
    out.write(
        '\n'.join(
            '\t'.join(map(str, [seq, 
                                Kds[other_run][seq], 
                                Kd_error[other_run][seq],
                                ABAs[other_run][seq],
                                ABA_error[other_run][seq]]))
            for seq in other_seqs_to_use
        )
    )

In [ ]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')